<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from IPython.display import HTML
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<a href="javascript:code_toggle()">Toggle code cells ON/OFF</a>.''')

In [3]:
plt.rcParams['figure.figsize'] = [14, 7]
plt.rcParams.update({'font.size': 19})

In [4]:
def F_W(x, a, b):
    return 1-np.exp(-((x/a)**b))

In [5]:
def psi(x, l, g, F, a, b):
    return g+(1-g-l)*F(x, a, b)

In [6]:
@interact
def psi_corr(threshold=(.01, 100, 1), slope=(.01, 20, 1), lapse=(0,.1,.01)):
    g=1/2
    x = np.linspace(0,100,200)
#     fig=plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
    _, ax = plt.subplots()
    ax.plot(x, 100 * psi(x, lapse, g, F_W, threshold, slope), linewidth=3)
    ax.plot([threshold, threshold], [50, 100*psi(threshold, lapse, g, F_W, threshold, slope)], color='green')
    ax.set(xlim=(0, 100), ylim=(50,100),
           xlabel="coherence",
           ylabel="percent correct",
           title="Weibull + lapses")

interactive(children=(FloatSlider(value=49.01, description='threshold', min=0.01, step=1.0), FloatSlider(value…

In [7]:
def F_L(x, a, b):
#     print('F_l called, ndim(x) is ', np.ndim(x))
    exp_arg = -b*(x-a)
    # set upper and lower bounds for argument to exponential
#     ub = 13
#     lb = -13
#     if np.ndim(exp_arg) == 0: # exp_arg is a scalar
#         exp_arg = ub if exp_arg > ub else exp_arg
#         exp_arg = lb if exp_arg < lb else exp_arg     
#     else:
#         exp_arg[exp_arg > ub] = ub
#         exp_arg[exp_arg < lb] = lb
    return 1/(1+np.exp(exp_arg))

In [8]:
@interact
def psi_choice(threshold=(-100, 100, 1), slope=(.01, .8, .05), lapse=(0,.1,.01)):
    g=lapse
    x = np.linspace(-100,100,200)
    _, ax = plt.subplots()
    ax.plot(x, 100 * psi(x, lapse, g, F_L, threshold, slope), linewidth=3)
    ax.plot([threshold, threshold], [0, 100*psi(threshold, lapse, g, F_L, threshold, slope)], color='green')
    ax.set(xlim=(-100, 100), ylim=(0,100),
           xlabel="signed coherence",
           ylabel="percent choose 'right'",
           title="Logistic + lapses")

interactive(children=(IntSlider(value=0, description='threshold', min=-100), FloatSlider(value=0.3600000000000…

In [9]:
@interact
def psi_choice_corr(thr_corr=(0.001,100,1),
                    slope_corr=(.01,20,1),
                    thr_choice=(-100, 100, 1), 
                    slope_choice=(.01, .8, .05), 
                    lapse=(0,.1,.01)):
    g = {'corr': 1/2, 'choice': lapse}
    x = {'choice': np.linspace(-100,100, 200), 'corr': np.linspace(0,100,200)}
    F = {'logistic': F_L, 'Weibull': F_W, 'choice': F_L, 'corr': F_W}
    threshold = {'choice': thr_choice, 'corr': thr_corr}
    slope = {'choice': slope_choice, 'corr': slope_corr}
    _, (ax1,ax2) = plt.subplots(1,2)
    ax = {'choice': ax2, 'corr': ax1}
    for case in ('choice', 'corr'):
#         print(case)
        ax[case].plot(x[case], 
                          100 * psi(x[case], lapse, g[case], F[case], 
                                    threshold[case], slope[case]), 
                      linewidth=3, 
                      color='red' if case == 'choice' else 'blue')
        if case == 'corr':
            c = 1/2 * (psi(abs(x[case]), lapse, g['choice'], F['logistic'], threshold['choice'], slope['choice']) + 
                       (1-psi(-abs(x[case]), lapse, g['choice'], F['logistic'], threshold['choice'], slope['choice'])))
#             print(type(c), c.shape)
            ax[case].plot(x[case], 100*c, color='red', linewidth=3)
        lylim = 45 if case == 'corr' else 0
        ax[case].plot([threshold[case], threshold[case]], 
                          [lylim, 100*psi(threshold[case], lapse, g[case], F[case], 
                                       threshold[case], slope[case])], color='green')
        ax[case].set(xlim=(0, 100) if case == 'corr' else (-100,100), 
                     ylim=(lylim, 100),
               xlabel="coherence" if case == 'corr' else "signed coherence",
               ylabel="percent choose 'right'" if case == 'choice' else 'percent correct',
                    title="choose 'right'" if case == 'choice' else 'percent correct')


interactive(children=(FloatSlider(value=49.001, description='thr_corr', min=0.001, step=1.0), FloatSlider(valu…